In [2]:
import os, cv2
from pycocotools.coco import COCO
from torchvision import io
import torch
import torch.nn as nn
import torchvision
import tensor
from torch.utils.data import Dataset, DataLoader
from torchvision.ops import nms


DATA_DIR = "C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete"
IMAGES_DIR = os.path.join(DATA_DIR, "Images")
MASKS_DIR = os.path.join(DATA_DIR, "Masks") 
ANNOTATIONS_DIR = os.path.join(DATA_DIR, "BoundingBoxs")

def get_annotations_file_path(image_filename):
    filename_without_extension = os.path.splitext(image_filename)[0]
    return os.path.join(ANNOTATIONS_DIR, f"{filename_without_extension}.json")

class CrackDataset:

  def __init__(self, root_dir, transform=None):
    self.root_dir = root_dir
    self.transform = transform
    self.coco = COCO(os.path.join(root_dir, 'annotations.json'))

  def __getitem__(self, idx):
    img = self.load_image(idx)
    target = self.load_targets(idx)
    if self.transform:
      img, target = self.transform(img, target)
    return img, target

  def __len__(self):
    return len(self.coco.imgs)

  def load_image(self, i):
  # get image ID, file name from coco
    img_id = self.coco.imgs[i]['id']  
    file_name = self.coco.imgs[i]['file_name']

  # load and return image
    img_path = os.path.join(self.root_dir, 'images', file_name)  
    img = cv2.imread(img_path) 
    return img, img_id

  def load_targets(self, img_id):
  # get ground truth annotations for image i
    ann_ids = self.coco.getAnnIds(imgIds = img_id)  
    annotations = self.coco.loadAnns(ann_ids)

  # parse bbox and mask from annotations
    boxes = []
    masks = []
    for ann in annotations:
      boxes.append(ann['bbox'])
    
    # load binary mask from COCO polygon area 
      mask = self.coco.annToMask(ann)  
      masks.append(mask)

  # construct targets dict   
    targets = {'boxes': boxes, 'labels': [], 'masks': masks}  
    return targets

dataset = CrackDataset(DATA_DIR)
dataloader = DataLoader(dataset)
train_set, val_set = torch.utils.data.random_split(dataset)

train_loader = DataLoader(train_set, batch_size=2)  
val_loader = DataLoader(val_set, batch_size=1)

model = torchvision.models.detection.maskrcnn_resnet50_fpn()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005) 

for epoch in range(30):

  model.train()
  for imgs, targets in train_loader:

    preds = model(imgs)
    loss = torch.sum(preds - targets) 

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    def evaluate(val_loader, model):

        model.eval()

        val_loss = 0

        with torch.no_grad():
          for imgs, targets in val_loader:

            outputs = model(imgs)
            loss = loss_fn(outputs, targets)  

            val_loss += loss.item()

        return val_loss / len(val_loader)

  model.eval()
  val_loss = evaluate(val_loader, model)

  print(f'Epoch {epoch}, Val Loss: {val_loss}')
  
model = torchvision.models.detection.maskrcnn_resnet50_fpn(num_classes=2)
params = [p for p in model.parameters() if p.requires_grad]

# Learning rate and optimizer
lr = 0.005
optimizer = torch.optim.SGD(params, lr=lr, 
                            momentum=0.9, weight_decay=0.0005)  

# Define anchor sizes as a list of lists
anchor_sizes = [[128, 128], [256, 256], [512, 512]] 

# Generate anchors for each size 
anchors = []
for size in anchor_sizes:

  # Generate anchors centered at reference point (0,0)
  anchor = torchvision.ops.boxes.anchor(size, device='cpu')  

  # Append to final anchors list
  anchors.append(anchor)

# Concatenate all anchors 
anchors = torch.cat(anchors, dim=0) 

# Print sample anchors
print(anchors[:6,:])

# For example:
tensor([[ -64., -64.,  64.,  64.],
        [-128., -128., 128., 128.],
        [ -256., -256., 256., 256.],
        [ -512., -512., 512., 512.],
        [ -512., -512., 512., 512.],
        [ -512., -512., 512., 512.]])

# 4. Define losses
criterion_cls = nn.CrossEntropyLoss()
criterion_bbox = nn.L1Loss()
criterion_mask = nn.BCELoss()

# 5. Compile losses
def loss_fn(outputs, targets):
  
  # Regression and classification losses
  loss_cls = criterion_cls(outputs["scores"], targets["labels"])
  loss_bbox = criterion_bbox(outputs["boxes"], targets["boxes"])  

  # Compute mask loss if present  
  has_mask = targets["masks"] is not None
  if has_mask:
    loss_mask = criterion_mask(outputs["masks"], targets["masks"])
    loss = loss_cls + loss_bbox + loss_mask
  else:
    loss = loss_cls + loss_bbox

  return loss

# Train loop

num_epochs = 30

for epoch in range(num_epochs):

  # Set to training mode
  model.train()  

  epoch_loss = 0
  
  for inputs, targets in train_loader:

    # Forward pass
    outputs = model(inputs)  

    # Compute loss
    loss = loss_fn(outputs, targets)  

    # Backward pass and optimize 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

  # Print status
  print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}')

def validate(val_loader, model):

  # Set to eval mode
  model.eval()  

  val_loss = 0

  for inputs, targets in val_loader:

    # Forward pass only
    outputs = model(inputs)

    # Compute validation loss 
    loss = loss_fn(outputs, targets)  
    val_loss += loss.item()

  print(f'Validation Loss: {val_loss/len(val_loader)}')
  
    # Validate at end of each epoch
  with torch.no_grad():
    validate(val_loader, model)

# Save model
torch.save(model.state_dict(), 'crack_detector.pth')

images, targets = next(iter(dataloader))
predictions = model(images)
keep = nms(predictions['boxes'], predictions['scores'])
masks = predictions['masks'][keep]

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (service.py, line 86)